# Introduction

The goal of this tutorial is to develop an approximate likelihood function for the Siegler addition model using Probability Density Approximation (PDA). The basic idea underlying PDA is that the likelihood can be approximated by simulating the data-generating process many times. Although this can be computationally intensive, it works even when an analytic likelihood function is difficult or unknown. 

One of the important lessons in this tutorial is how to speed up PDA. A speed up can be accomplished with at least two techniques. One technique involves caching results to eliminate redundant computations. For example, if a set of responses are identical and independently distributed under the model, a set of simulated responses can be cached and reused to approximate the likelihood function. Second, independent components of the model can be simulated on seperate processors. In many situations, there are several ways to delegate work to multiple processors. To achieve maximum efficiency, it is important to (1) keep all processors occupied as much as possible, and (2) minimize the proportion of time attributable to overhead. Running a simulation on parallel processors requires some overhead to manage the jobs and to transfer data to and from the processors. A simulation will run relatively fast if all processors are busy and overhead is small relative to the processing time of the simulation. 

# Siegler Addition Task

The Siegler addition task is based on a study of childrens' arithmatic ability conducted by Siegler (1984). In the task, children are asked to sum two verbally presented numbers (e.g. 2 + 2), and provide the solution verbally. If the participant does not know the solution, he or she responds "I don't know".  Each block consists of the following problems:

- 1 + 1
- 1 + 2
- 1 + 3
- 2 + 2
- 2 + 3 
- 3 + 3

The block is repeated five times in the present simulation.


# Siegler Addition Model

## Overview

On each trial, the Siegler addition model proceeds through a deterministic chain of production rules that build the problem representation in the imaginal buffer, retrieve the answer, and respond. For each number, the model fires a production rule that listens to the number, retrieves the number from declarative memory, and adds the number to a problem representation chunk in the imaginal buffer. The resulting chunk is used as a retrieval request in the next production rule to obtain the answer from declarative memory. When the next production rule fires, it "harvests" the answer, which entails assigning the answer to a sum slot in the problem representation chunk. Finally, the model fires a production rule to vocalize the answer and merge the problem representation into declarative memory.




## Declarative memory

Declarative memory $M$ is populated with 35 chunks representing addition facts. As an example, consider the chunk $m$:

\begin{align}
\mathbf{c}_m = \{\rm (addend1,2),(addend2,2), (sum,4)\}
\end{align}

Each chunk contains the following three slots: $Q = \{\textrm{addend1},\textrm{addend2}, \textrm{sum}\}$ .The full set of 35 addition facts is generated by permuting integers ranging from 0 to 5 for addend1 and addend2, such that the sum is less 10. Formally, this set is defined as:

\begin{align}
M = \{ \forall_{i,j} \rm [(addend1, j), (addend2, i), (sum, i + j)] : i,j \in \{0,1,\dots 5\}, i + j < 10  \}
\end{align}



## Retrieval Request

The details of the problem are encoded through the aural module where they are transferred and stored in a chunk $\mathbf{c}_s$ located in the imaginal buffer. A retrieval request on trial $i$ is formed from the slots in $\mathbf{c}_s$, which is defined as

\begin{align}
\mathbf{r}_i = \{(\rm addend1, c_s(addend1), (\rm addend2, c_s(addend2) \}
\end{align}

where $Q_r = \{\rm addend1, addend2\}$ is the set of slots for $\mathbf{r}_i$. 

## Activation

Activation for chunk $\mathbf{c}_m$ is defined as:

\begin{align}
a_m = \textrm{blc}_m + \rho_m + \epsilon_m
\end{align}

where $\textrm{blc}_m$ represents degree of prior practice, $\rho_m$ is partial matching activation and $\epsilon \sim \rm normal(0,s)$. Unlike previous models, blc is indexed by $m$ to reflect the fact that its value depends on the chunk. The rationale for allowing blc to vary as function of $m$ is due to the fact that children have more practice with addition problems in which the sum is less than five, and thus, perform better with these problems. Based on this fact, we can define blc with the following piecewise equation:

\begin{align}\label{eq:penalty_activation_siegler}
\textrm{blc}_m  = \begin{cases}
    .65 & \text{if } c_m(\rm sum) < 5 \\
    0 & \text{otherwise} \\
  \end{cases}\
\end{align}


The model uses the following graded mismatch penalty function

\begin{align}
\rho_m  = -\textrm{sim} \times \delta \sum_{q \in Q_r}| c_m(q) - r(q) | 
\end{align}

where sim = .1 is a similarity scalar. The penalty function applies greater penalty when the slot values differ by a larger degree. 


## Response Mapping

Due to the partial matching mechanism, a given response could be associated with one of several possible chunks. For example, the response $y = 4$ could have resulted from retrieving any chunk that represents the following addition facts:
- $2+2$
- $1+3$
- $3+1$ 
- $4+0$
- $0+4$

In order to account for this in the likelihood function, we define the set $R_i$, which consists of all possible chunks associated with response $y_i$ on trial $i$. Formally, this is defined as:


\begin{equation}
R_i = \{\mathbf{c}_m \in M : c_m(\rm sum) = \textrm{y_i} \} \textrm{ for } y_i \neq \emptyset
\end{equation}

with a response indicating that the sum is unknown ($y_i =\emptyset$) when no chunk is retrieved. 

## Response Probability

In actuality, the approximate response probability is known. To review, let $y_i \in \{1,2,\dots,\emptyset\}$ by the response on trial $i$, where $\emptyset$ represents a retrieval failure response. The approximate probability is given by the expression:

\begin{align}
     \Pr(Y_i = y_i \mid \delta ; \mathbf{r}_i) \approx \frac{\sum_{\mathbf{c}_m \in R_i} e^{\frac{\mu_m}{\sigma}}}{\sum_{\mathbf{c}_k \in M} e^{\frac{\mu_k}{\sigma}} + e^{\frac{\mu_{m^\prime}}{\sigma}}}
\end{align}

However, let's assume that we do not have an expression for the probability or approximate probability. In that case, we can generate data from the model to approximate the true probability. Let $X = \{x_1,x_1,\dots, x_{n_s}\}$ be a set of $n_s$ simulated data. Thus, the approximate probability of observing $y_i$
\begin{equation}
 \Pr(Y_i = y_i \mid \delta ; \mathbf{r}_i) \approx \frac{1}{n_s} \sum_{i=1}^{n_s} I(x_i, y_i)
\end{equation}
where indicator function $I(x_i,h)$ yields 1 if the inputs are equal and 0 otherwise. As $n_s$ increases, the approximation becomes more accurate and converges on the analytic likelihood in the limit.

## Assumptions

The following is a summary of the model's assumptions:

1. Retrieval failures are possible
2. Retrieval probabilities are independent
3. No learning occurs
4. Activation is a decreasing function of the difference between the slot values of a chunk and and the slot values of the retrieval request
5. Errors are due to retrieving the wrong information rather than encoding the information incorrectly

## Parallel Processes

Sizable performance gains can be achieved by running code independently on multiple processors. To run parallel code, we will load the `Distributed` package and call the function `addprocs`. If your computer does not have 4 processors, you can change the argument below. 

In [18]:
# set current directory to directory that contains this file
cd(@__DIR__)
using Pkg, Distributed
# activate tutorial environment
Pkg.activate("../../..")
# load packages
using StatsPlots, DataFrames, MCMCChains
using ACTRModels, Distributions, DifferentialEvolutionMCMC
# specify the number of processors
addprocs(4)

  Activating project at `~/.julia/dev/ACTRTutorial`
┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1423
┌ Info: Precompiling MCMCChains [c7f686f2-ff18-58e9-bc7b-31028e88f75d]
└ @ Base loading.jl:1423
┌ Info: Precompiling ACTRModels [c095b0ea-a6ca-5cbd-afed-dbab2e976880]
└ @ Base loading.jl:1423
┌ Info: Precompiling DifferentialEvolutionMCMC [607db5a9-722a-4af8-9a06-1810c0fe385b]
└ @ Base loading.jl:1423


4-element Vector{Int64}:
 2
 3
 4
 5

We can use the function `nprocs` to verify that the additional processors where successfully added. The return value should be 1 (main processor) plus the number of processors specified in `addprocs`.

In [2]:
nprocs()

1

### @everywhere macro

We will use the `@everywhere` macro to load the required code and packages to each processor. When the `@everywhere` macro prefixes a line of code, that code is loaded to each parallel worker. In cases where multiple lines of code need to be added to the processors, we can wrap the code in a `begin` block and apply `@everywhere` to the `begin` block. In the following block, we added the path to our environment, the required packages, the required code, and seed for the random number generator. 

In [19]:
@everywhere begin
  # path to the tutorial environment
  push!(LOAD_PATH, "../../..")
  # required packages
  using ACTRModels, Distributions, DifferentialEvolutionMCMC
  # required model code
  include("Siegler_Model_Choice_PDA.jl")
  include("../../../Utilities/Utilities.jl")
  # seed the random number generator
  Random.seed!(774145)
end

      From worker 4:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ACTRModels [c095b0ea-a6ca-5cbd-afed-dbab2e976880]
      From worker 4:	│   exception = Required dependency SequentialSamplingModels [0e71a2a6-2b30-4447-8742-d083a85e82d1] failed to load from a cache file.
      From worker 4:	└ @ Base loading.jl:1132
      From worker 2:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ACTRModels [c095b0ea-a6ca-5cbd-afed-dbab2e976880]
      From worker 2:	│   exception = Invalid input in module list: expected SequentialSamplingModels.
      From worker 2:	└ @ Base loading.jl:1132
      From worker 3:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for ACTRModels [c095b0ea-a6ca-5cbd-afed-dbab2e976880]
      From worker 3:	│   exception = Invalid input in module list: expected SequentialSamplingModels.
      From worker 3:	└ @ Base loading.jl:1132
      From worker 5:	┌ 

### pmap

`pmap` is a parallel version of `map`. We will use this later to simulate the model in parallel. Recall that the function `map` simply broadcasts or applies arguments to a function. As a simple example, we will create a function `hello` that prints "hello" an apply it to `map` ten times:

In [4]:
@everywhere hello() = println("hello")
map(_->hello(), 1:10);

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


Note that `_->` is used when a function does not accept any arguments. `pmap` is simple and works the same way as `map`. The worker (i.e. processor) id is printed automatically along with the print statement:

In [5]:
pmap(_->hello(), 1:10);

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


Importantly, manual management of the jobs is not necessary. `pmap` schedules the next job as soon as a worker is available. 

# Generate Data

In the code block below,  we will define a function to generate simulated data from the model. The `simulate` function accepts the following arguments:

- stimuli: a `NamedTuple` of addition problems. 
- fixed_parms: a `NamedTuple` of fixed parameters
- args...: list of keyword arguments for estimated parameters

In the annotated code below, `simulate` calls two other functions: `initialize_model` and `simulate_trial`. The function `initialize_model` returns a new ACT-R model object based on `fixed_parms` and other paramenters specified in `args...`. The function `simulate_trial` generates a response for a given stimulus. The reason for creating a separate function for simulating a single trial will become appearent later when we need to generate many samples for a given stimulus in order to approximate the likelihood. 

In [6]:
function simulate(stimuli, fixed_parms; args...)
    N = length(stimuli)
    # initialize data
    data = Array{NamedTuple,1}(undef, N)
    # initialize ACT-R model 
    actr = initialize_model(fixed_parms; args...)
    # simulate each trial with stimulus s
    for (i,s) in enumerate(stimuli)
        response = simulate_trail(actr, s)
        data[i] = (s..., resp=response)
    end
    return data
end

function initialize_model(fixed_parms; args...)
   # populate memory with addition facts
   chunks = populate_memory()
   # set blc parameters for each chunk
   set_baselevels!(chunks)
   # add parameters and chunks to declarative memory
   memory = Declarative(;memory=chunks)
   # add declarative memory to ACTR object
   return actr = ACTR(;declarative=memory, fixed_parms..., args...)
end

function simulate_trail(actr, stimulus)
    # retrieve chunk using stimulus as retrieval request
    chunk = retrieve(actr; stimulus...)
    # return sum slot if retrieved, -100 for retrieval failure
    return isempty(chunk) ? -100 : chunk[1].slots.sum
end

simulate_trail (generic function with 1 method)

The code block below generates 5 blocks of data using parameters $\delta = 16$. The data for each trial is a `NamedTuple` consisting of 

- num1: the first addend of the problem
- num2: the second addedn of the problem
- resp: the response where -100 indicates "I don't know"
- N: the number of instances of a specific problem-response combination

In [7]:
n_blocks = 5
parms = (δ=16.0,)
fixed_parms = (s=.5, τ=-.45, mmp = true,noise = true,mmp_fun = sim_fun,ter = 2.05)
stimuli = [(num1 = 1,num2 = 1), (num1 = 1,num2 = 2), (num1 = 1,num2 = 3), (num1 = 2,num2 = 2),
    (num1 = 2,num2 = 3), (num1 = 3,num2 = 3)]
temp = mapreduce(_->simulate(stimuli, fixed_parms; parms...), vcat, 1:n_blocks)
temp = unique_data(temp)
sort!(temp)
data = map(x->filter(y->y.num1==x.num1 && y.num2==x.num2 , temp), stimuli);

LoadError: UndefVarError: sim_fun not defined

The most time consuming part of approximating the likelihood function is generating simulated data. Any measure to cache and reuse simulated data will improve performance. In the code block above, two measures were taken to improve efficiency. First, we used `unique` to count the number of unique responses. For example, there were four responses of $2$ to $1 + 1$:

In [8]:
data[1][1]

LoadError: UndefVarError: data not defined

In such cases, we can multiply the approximate log probability of responding $2$ to the question $1 + 1$ by 4. The second effeciency measure involves grouping data according to stimuli so that the simulated data can be reused, even if the responses are different. Consider the following:

In [9]:
data[1]

LoadError: UndefVarError: data not defined

The first sub-vector contains all responses to the question $1 + 1$, which is composed of 4 responses of 2 and 1 response of 3. By grouping the data according to stimuli, we can use the same simulated data when approximating the response probabilities. In this example, we will first determine the relative frequency of $2$ and mutiply the log probability by 4, and then determine the relative frequency of $3$ using the same simulated data. 

## Define Likelihood Function

The function `loglike` is the primary function responsible for computing the log likelihood of data and is passed to the MCMC sampler. `loglike` requires the following inputs:

- data: a vector of vectors of `NamedTuples` grouped by stimulus representing the data
- stimuli: a vector of stimuli 
- fixed_parms: a `NamedTuple` of fixed parameters and settings
- $\delta$: mismatch penalty parameter
- n_sim: a keyword argument for the number of simulations for approximating the likelihood function

`loglike` performs the following operations: first, an ACT-R model object is created with the parameters in `fixed_parms` and $\delta$. Next, a temporary function is created which accepts a data vector and a corresponding stimulus. The function `pmap` broadcasts each pair in `data` and `stimuli` to a separate processor, where the log likelihood is computed in `loglike_trial`.  Finally, the sum of all log likelihoods is returned.

The function `loglike_trial` computes the approximate log likelihood for each trail. Techically, `loglike_trial` computes the log likelihood for trials with the same stimulus. It requires the following inputs:

- actr: an ACT-R model object
- stimuli: a vector of stimuli 
- fixed_parms: a `NamedTuple` of fixed parameters and settings
- data: a vector of `NamedTuples` of data for a given stimulus
- n_sim: the number of simulations for approximating the likelihood function

`loglike_trial` first generates `n_sim` responses for a given stimulus. In the for loop, the approximate log probability is computed from the simulated responses. The log likelihood of each response in $d$ is weighted by the number of responses of the same type. For example, if there were 3 responses of 2 to the question $1+1$, the log likelihood will be weighted by 3. 

In [10]:
function loglike(data, stimuli, fixed_parms, δ; n_sim=1000)
    # initialize the model with fixed_parms and δ
    actr = initialize_model(fixed_parms; δ)
    # temporary function for loglike_trial with two arguments
    f(s, x) = loglike_trial(actr, s, x, n_sim)
    # map the corresponding elements of stimuli and data to each processor
    LLs = pmap(f, stimuli, data)
    # sum the log likelihood. (Annotate return type because pmap is not type-stable)
    return sum(LLs)::Float64
end

function loglike_trial(actr, stimulus, data, n_sim) 
    # simulate trial n_sim times 
    preds = map(_->simulate_trail(actr, stimulus), 1:n_sim)
    LL = 0.0
    # compute approximate log likelihood for each trial in data 
    for d in data 
        p = max(mean(preds .== d.resp), 1/n_sim)
        # multiply log likelihood by number of replicates N
        LL += log(p)*d.N
    end
    return LL
end

loglike_trial (generic function with 1 method)

## Define Model
The following summaries the prior distributions and the likelihood. 

\begin{align}
\delta \sim \textrm{Normal}(16,8) 
\end{align}


\begin{align}
 \theta_i  \approx \frac{1}{n_s} \sum_{i=1}^{n_s} I(x_i, y_i)
\end{align}


\begin{align}
y_i \sim \textrm{Bernoulli}(\theta_i)
\end{align}

Next, we will set up the model and MCMC sampler. First, we specify the prior distributions and the boundaries for each parameter. Next, a model object is defined. The optional positional objects are passed to `loglike`. The keyword arguments correspond to the prior distribution, the `loglike` function of the model, and the `data`. 

In [11]:
# prior distribution over δ
priors = (
    δ = (Normal(16, 8),),
)

# boundaries for δ
bounds = ((-Inf,Inf),)

# model object.
model = DEModel(stimuli, fixed_parms; priors, model=loglike, data);

LoadError: UndefVarError: Normal not defined

## Estimate Parameters

We will estimate the parameter $\delta$ using Differential Evolution MCMC because it can be used with approximate likelihood functions. We will run one group of eight particles for 2,000 iterations and discard the first 1,000 warmup samples. 

In [12]:
de = DE(;bounds, burnin=1000, priors, n_groups=1, Np=8)
n_iter = 2000
chain = sample(model, de, n_iter, progress=true)

LoadError: UndefVarError: bounds not defined

## Results

### Diagnostics

The first panel for each plot shows acceptable mixing between the eight chains. Furthermore, $\hat{r} \approx 1$ for $\delta$, indicating that the chains converged.  In the second panel for each plot, the auto-correlation between successive samples drops close to zero after a lag of about 10-15, which is typical for Differential Evolution MCMC, particularly with approximation error in the likelihood function. 
### Posterior Distributions 

The posterior distributions in the third panel of each plot encompass the data generating value ($\delta = 16$), indicating acceptable parameter recovery. Although the posterior distributions are somewhat wide, their standard deviations are smaller compared to the standard deviations of the prior distributions. 



In [13]:
pyplot()
font_size = 12
ch = group(chain, :δ)
p1 = plot(ch, xaxis=font(font_size), yaxis=font(font_size), seriestype=(:traceplot),
  grid=false, size=(250,100), titlefont=font(font_size))
p2 = plot(ch, xaxis=font(font_size), yaxis=font(font_size), seriestype=(:autocorplot),
  grid=false, size=(250,100), titlefont=font(font_size))
p3 = plot(ch, xaxis=font(font_size), yaxis=font(font_size), seriestype=(:mixeddensity),
  grid=false, size=(250,100), titlefont=font(font_size))
pcτ = plot(p1, p2, p3, layout=(3,1), size=(800,600))

LoadError: UndefVarError: pyplot not defined

The pooled density plot provides a smoother depiction of the posterior distribution of $\delta$.

In [14]:
pooleddensity(ch, grid=false, xaxis=font(font_size), yaxis=font(font_size), size=(800,250))

LoadError: UndefVarError: font not defined

### Posterior Predictive Distribution

The code block below generates the posterior distribution of of response probabilities for each of the six problems. A similar pattern emerges across all six problems: the correct answer is the most likely response at $\approx .6$ and the probability of a response becomes less and less likely as it deviates further from the correct response. 


In [15]:
preds = posterior_predictive(x -> simulate(stimuli, parms; x...), chain, 1000)
preds = vcat(vcat(preds...)...)
df = DataFrame(preds)
p5 = response_histogram(df, stimuli; size=(800,400))

LoadError: UndefVarError: posterior_predictive not defined

# References

Siegler, R. S., & Shrager, J. (1984). Strategy choices in addition and
subtraction: How do children know what to do? In C. Sophian (Ed.),
The origins of cognitive skills (pp. 229-293). Hillsdale, NJ: Erlbaum

Turner, B. M., & Van Zandt, T. (2012). A tutorial on approximate Bayesian computation. Journal of Mathematical Psychology, 56(2), 69-85.

Turner, B. M., & Sederberg, P. B. (2014). A generalized, likelihood-free method for posterior estimation. Psychonomic bulletin & review, 21(2), 227-250.

Turner, B. M., Sederberg, P. B., Brown, S. D., & Steyvers, M. (2013). A method fo